# 🔐 Data Anonymization with LLMs - End-to-End Demo

This notebook demonstrates the modular framework for text anonymization.

**Available methods:**
- **EDA** (Easy Data Augmentation): Baseline techniques using WordNet
- **KNEO** (Knowledge-based Neighbor Operation): Embeddings (GloVe/fastText)
- **LLM**: Language models via Ollama (gemma2, llama3.2, mistral, etc.)

**To change configuration:**
Edit the `configs/config.yaml` file to change models, datasets, and parameters.

## 1. 🚀 Setup

In [ ]:
# Install dependencies (first time only)
# !pip install -r ../requirements.txt
# !python -m spacy download en_core_web_sm

In [ ]:
import sys
import warnings
warnings.filterwarnings('ignore')

# Add src to path
sys.path.insert(0, '../src')

# Import framework modules
from eda_anonymizer import EDAAnonymizer
from kneo_anonymizer import KNEOAnonymizer
from llm_anonymizer import OllamaAnonymizer, PROMPT_TEMPLATES
from metrics import AnonymizationMetrics
from utils import (
    load_config, 
    load_all_datasets, 
    set_all_seeds,
    create_output_dir,
    save_anonymized_dataset,
    save_metrics,
    print_comparison_table
)

# Import visualization libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Setup visualization
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_colwidth', None)

print("✅ Libraries imported successfully")

from src.utils import load_config, load_all_datasets
from src.eda_anonymizer import EDAAnonymizer
from src.kneo_anonymizer import KNEOAnonymizer
from src.llm_anonymizer import OllamaAnonymizer
from src.metrics import AnonymizationMetrics

In [ ]:
# Load configuration from YAML file
config = load_config('../configs/config.yaml')

# Display main configuration
print("\n📋 CURRENT CONFIGURATION:")
print(f"   EDA alphas: SR={config['eda']['alpha_sr']}, RI={config['eda']['alpha_ri']}, RS={config['eda']['alpha_rs']}, RD={config['eda']['alpha_rd']}")
print(f"   KNEO: model={config['kneo']['embedding_model']}, k={config['kneo']['k']}")
print(f"   LLM: model={config['llm']['model_name']}, temp={config['llm']['temperature']}")
print(f"   Metrics: SBERT={config['metrics']['sbert_model']}")

### 1.2 Load Dataset

In [ ]:
# Define sample size for quick testing (None = full dataset)
SAMPLE_SIZES = {
    'train': 100,       # Change to None to use all data
    'validation': 50,
    'test': None
}

# Load datasets
datasets = load_all_datasets(
    config, 
    base_dir='../data',
    sample_sizes=SAMPLE_SIZES
)

# Extract for convenience
if 'train' in datasets:
    train_texts, train_labels = datasets['train']
    print(f"\n📊 Training set: {len(train_texts)} samples")

if 'validation' in datasets:
    val_texts, val_labels = datasets['validation']
    print(f"📊 Validation set: {len(val_texts)} samples")

In [ ]:
# Display some examples
print("\n🔍 Examples from dataset:\n")
for i in range(min(5, len(val_texts))):
    print(f"[{val_labels[i]}] {val_texts[i][:100]}..." if len(val_texts[i]) > 100 else f"[{val_labels[i]}] {val_texts[i]}")

---

## 2. 📝 Metodo 1: Easy Data Augmentation (EDA)

EDA applica 4 tecniche di augmentation:
- **SR** (Synonym Replacement): Sostituisce parole con sinonimi da WordNet
- **RI** (Random Insertion): Inserisce sinonimi casuali
- **RS** (Random Swap): Scambia posizioni di parole
- **RD** (Random Deletion): Elimina parole casuali

In [ ]:
# Initialize EDA Anonymizer
eda = EDAAnonymizer()

# Apply anonymization
print("🔄 Applying EDA...")
anonymized_eda = eda.anonymize_batch(
    val_texts,
    alpha_sr=config['eda']['alpha_sr'],
    alpha_ri=config['eda']['alpha_ri'],
    alpha_rs=config['eda']['alpha_rs'],
    alpha_rd=config['eda']['alpha_rd'],
    show_progress=True
)

print(f"\n✅ EDA completed: {len(anonymized_eda)} sentences anonymized")

In [ ]:
# Show examples
print("📝 EDA anonymization examples:\n")
print("-" * 80)
for i in range(min(5, len(val_texts))):
    print(f"\n🔹 ORIGINAL:  {val_texts[i]}")
    print(f"🔸 EDA:       {anonymized_eda[i]}")
    print("-" * 80)

---

## 3. 🧠 Metodo 2: KNEO (Knowledge-based Neighbor Operation)

KNEO usa embeddings pre-addestrati per sostituire parole con vicini semantici.

**Modelli disponibili:**
- `glove-wiki-gigaword-50/100/200/300` - GloVe embeddings
- `fasttext-wiki-news-subwords-300` - FastText (migliore per testi rumorosi)

In [ ]:
# Initialize KNEO Anonymizer
kneo = KNEOAnonymizer(
    embedding_model=config['kneo']['embedding_model'],
    verbose=True
)

In [ ]:
# Apply anonymization
print("\n🔄 Applying KNEO...")
anonymized_kneo = kneo.anonymize_batch(
    val_texts,
    k=config['kneo']['k'],
    strategy=config['kneo']['strategy'],
    show_progress=True
)

print(f"\n✅ KNEO completed: {len(anonymized_kneo)} sentences anonymized")

# Cache statistics
cache_stats = kneo.get_cache_stats()
print(f"💾 Cache: {cache_stats['cache_size']} unique words processed")

In [ ]:
# Show examples
print("📝 KNEO anonymization examples:\n")
print("-" * 80)
for i in range(min(5, len(val_texts))):
    print(f"\n🔹 ORIGINAL:  {val_texts[i]}")
    print(f"🔸 KNEO:      {anonymized_kneo[i]}")
    print("-" * 80)

---

## 4. 🤖 Method 3: LLM with Ollama

Use local LLM models via Ollama for more sophisticated anonymization.

### Ollama Setup (if not already installed)

```bash
# Run the setup script
bash scripts/setup_ollama.sh

# Or manually:
# macOS: brew install ollama
# Linux: curl -fsSL https://ollama.ai/install.sh | sh

# Start the server
ollama serve

# Download a model (in another terminal)
ollama pull gemma2:2b    # Small and fast
ollama pull llama3.2     # Meta's latest
ollama pull mistral      # Good balance
```

In [ ]:
# Try connecting to Ollama
try:
    llm = OllamaAnonymizer(
        model_name=config['llm']['model_name'],
        base_url=config['llm']['base_url'],
        temperature=config['llm']['temperature'],
        max_tokens=config['llm']['max_tokens'],
        prompt_style="paraphrase",
        verbose=True
    )
    
    OLLAMA_AVAILABLE = True
    print("\n✅ Ollama connected and ready!")
    
except Exception as e:
    OLLAMA_AVAILABLE = False
    print(f"\n⚠️  Ollama not available: {e}")
    print("   Skip this section if you don't have Ollama installed.")

In [ ]:
# Apply LLM (only if available)
if OLLAMA_AVAILABLE:
    # Use a smaller subset for LLM (it's slower)
    llm_sample_size = min(20, len(val_texts))
    llm_texts = val_texts[:llm_sample_size]
    llm_labels = val_labels[:llm_sample_size]
    
    print(f"\n🔄 Applying LLM ({llm_sample_size} samples)...")
    
    anonymized_llm = llm.anonymize_batch(
        llm_texts,
        labels=llm_labels,
        show_progress=True
    )
    
    print(f"\n✅ LLM completed: {len(anonymized_llm)} sentences anonymized")
else:
    anonymized_llm = None

In [ ]:
# Show LLM examples
if anonymized_llm:
    print("📝 LLM anonymization examples:\n")
    print("-" * 80)
    for i in range(min(5, len(anonymized_llm))):
        print(f"\n🔹 ORIGINAL:  {llm_texts[i]}")
        print(f"🔸 LLM:       {anonymized_llm[i]}")
        print("-" * 80)

---

## 5. 📊 Evaluation with Metrics

Evaluate results with 4 metrics:

| Metric | Category | Goal |
|---------|-----------|----------|
| Levenshtein Ratio | Irreversibility | ↓ Lower = better |
| Jaccard Similarity | Irreversibility | ↓ Lower = better |
| Cosine Similarity | Semantic Utility | ↑ Higher = better |
| NER Score | Anonymization | ↑ Higher = better |

In [ ]:
# Initialize metrics module
metrics = AnonymizationMetrics(
    sbert_model=config['metrics']['sbert_model'],
    spacy_model=config['metrics']['spacy_model'],
    verbose=True
)

In [ ]:
# Evaluate EDA
print("\n📊 Evaluating EDA...")
eda_results = metrics.evaluate_all(val_texts, anonymized_eda, show_progress=True)

In [ ]:
# Evaluate KNEO
print("\n📊 Evaluating KNEO...")
kneo_results = metrics.evaluate_all(val_texts, anonymized_kneo, show_progress=True)

In [ ]:
# Evaluate LLM (if available)
if anonymized_llm:
    print("\n📊 Evaluating LLM...")
    llm_results = metrics.evaluate_all(llm_texts, anonymized_llm, show_progress=True)
else:
    llm_results = None

### 5.1 Compare Results

In [ ]:
# Collect all results
all_results = {
    'EDA': eda_results,
    'KNEO': kneo_results
}

if llm_results:
    all_results['LLM'] = llm_results

# Print comparison table
print_comparison_table(all_results)

In [ ]:
# Create DataFrame for analysis
df_results = pd.DataFrame(all_results).T
df_results.index.name = 'Method'
df_results = df_results.round(4)
display(df_results)

### 5.2 Visualization

In [ ]:
# Bar chart for comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

metrics_names = ['levenshtein_ratio', 'jaccard_similarity', 'cosine_similarity', 'ner_score']
metric_labels = ['Levenshtein Ratio ↓', 'Jaccard Similarity ↓', 'Cosine Similarity ↑', 'NER Score ↑']
colors = ['#e74c3c', '#e74c3c', '#27ae60', '#27ae60']

methods = list(all_results.keys())

for idx, (metric, label, color) in enumerate(zip(metrics_names, metric_labels, colors)):
    ax = axes[idx // 2, idx % 2]
    values = [all_results[m][metric] for m in methods]
    
    bars = ax.bar(methods, values, color=color, alpha=0.8, edgecolor='black')
    ax.set_title(label, fontsize=12, fontweight='bold')
    ax.set_ylim(0, 1)
    
    for bar, val in zip(bars, values):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
                f'{val:.3f}', ha='center', va='bottom', fontsize=10)

plt.suptitle('Anonymization Metrics Comparison', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---

## 6. 💾 Save Results

In [ ]:
# Create output directory
output_dir = create_output_dir(config)
print(f"📁 Output directory: {output_dir}")

In [ ]:
# Save anonymized datasets
if config['output']['save_anonymized']:
    save_anonymized_dataset(anonymized_eda, val_labels, output_dir, 'eda')
    save_anonymized_dataset(anonymized_kneo, val_labels, output_dir, 'kneo')
    
    if anonymized_llm:
        save_anonymized_dataset(anonymized_llm, llm_labels, output_dir, 'llm')

In [ ]:
# Save metrics
if config['output']['save_metrics']:
    save_metrics(all_results, output_dir, 'comparison')

---

## 7. 🔬 Advanced Experiments

In [ ]:
# Test different alpha configurations for EDA
alpha_configs = [
    {'name': 'Light', 'sr': 0.1, 'ri': 0.1, 'rs': 0.1, 'rd': 0.1},
    {'name': 'Medium', 'sr': 0.3, 'ri': 0.3, 'rs': 0.3, 'rd': 0.3},
    {'name': 'Aggressive', 'sr': 0.5, 'ri': 0.5, 'rs': 0.5, 'rd': 0.5},
]

eda_comparison = {}
test_texts = val_texts[:30]

for cfg in alpha_configs:
    print(f"\n🔄 Testing EDA {cfg['name']}...")
    anon = eda.anonymize_batch(
        test_texts, 
        alpha_sr=cfg['sr'], alpha_ri=cfg['ri'], 
        alpha_rs=cfg['rs'], alpha_rd=cfg['rd'],
        show_progress=False
    )
    results = metrics.evaluate_all(test_texts, anon, show_progress=False)
    eda_comparison[f"EDA-{cfg['name']}"] = results
    print(f"   Cosine: {results['cosine_similarity']:.3f}, NER: {results['ner_score']:.3f}")

print_comparison_table(eda_comparison)

---

## 8. 📝 Conclusions

### Summary of methods:

| Method | Pros | Cons | When to use |
|--------|-----|--------|---------------|
| **EDA** | Fast, offline | Less sophisticated | Baseline, large datasets |
| **KNEO** | Good semantics | Requires embeddings | Noisy datasets |
| **LLM** | High quality | Slow, requires GPU | Small datasets, max quality |

### How to replicate experiments:

1. Edit `configs/config.yaml` to change parameters
2. Add your datasets in `data/`
3. Run this notebook from the beginning

In [ ]:
print("\n" + "="*60)
print("🎉 DEMO COMPLETED!")
print("="*60)
print("\nNext steps:")
print("1. Edit configs/config.yaml for your experiments")
print("2. Add your datasets in data/")
print("3. Use scripts/run_baseline.py for full runs")
print("4. Use scripts/run_llm.py for LLM on large datasets")